In [1]:
import networkx as nx
from main import Board, GameGraph

In [4]:
N = 4
board = Board(N)
gg = GameGraph(board)

In [6]:
gg.play_random_game()

New game.

Player 1 inserts tile ██ at the bottom of column 0.
________
________
________
██______

Player 2 inserts tile ██ to the right of row 1.
________
______██
________
██______

Player 1 places tile ██ at column 0, row 0.
██______
______██
________
██______

Player 2 places tile ░░ at column 2, row 1.
██______
____░░██
________
██______

Player 1 inserts tile ██ to the right of row 0.
______██
____░░██
________
██______

Player 2 places tile ██ at column 1, row 1.
______██
__██░░██
________
██______

Player 1 inserts tile ░░ to the left of row 2.
______██
__██░░██
░░______
██______

Player 2 inserts tile ░░ to the right of row 2.
______██
__██░░██
______░░
██______

Player 1 inserts tile ██ at the bottom of column 1.
__██__██
____░░██
______░░
████____

Player 2 inserts tile ░░ to the right of row 3.
__██__██
____░░██
______░░
██____░░

Player 1 inserts tile ██ to the right of row 0.
██__████
____░░██
______░░
██____░░

Player 2 inserts tile ██ at the top of column 0.
██__████
█